In [3]:
import argparse
import statistics
import pandas as pd
import numpy as np

import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from sklearn.metrics import classification_report

def compute_metrics(data, y_true, y_pred, probs_emotions, id_prompt, output_file):
    print(classification_report(y_true, y_pred))
    data[id_prompt] = y_pred
    data['prob_'+id_prompt] = probs_emotions
    data.to_excel(output_file, index=False)

def compute_entailment(data, template_emo_name, template_expr_emo, template_feels_emo, prompts, output_file):

    print("Loading model...")

    model = AutoModelForSequenceClassification.from_pretrained('microsoft/deberta-v2-xlarge-mnli')
    tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v2-xlarge-mnli')

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    print("Predicting data using", prompts, "...")

    text_list = data['TEXT'].tolist()
    label_list = data['SENTIMENT'].tolist()
    unique_labels = sorted(list(set(label_list)))

    for id_prompt in prompts:
        probs_emotions = []
        y_true = []
        y_pred = []
        for i, text in enumerate(text_list):
            premise = text
            dict_emo_conf = {x: {} for x in unique_labels}
            for label in unique_labels:
                probs_syn = []
                with torch.no_grad():
                    if id_prompt == "emo_s":
                        template = template_emo_name
                        context = ''
                    if id_prompt == "expr_s":
                        template = template_expr_emo
                        context = 'This text expresses '
                    elif id_prompt == "feels_s":
                        template = template_feels_emo
                        context = 'This person feels '
                    for syn_prompt in template[label]:
                        x = tokenizer.encode(str(premise), str(context + syn_prompt), return_tensors='pt',truncation_strategy='only_first')
                        x = x.to(device)
                        logits = model(x)[0]
                        entail_contradiction_logits = logits[:,[0,2]]
                        prob_label_is_true = entail_contradiction_logits.softmax(dim=1)[:,1]
                        probs_syn.append(prob_label_is_true.cpu().detach().numpy().tolist()[0])
                    mean = statistics.mean(probs_syn)
                dict_emo_conf[label] = mean
            probs_emotions.append(list(dict_emo_conf.values()))
            final_emo = max(dict_emo_conf, key=dict_emo_conf.get)
            y_pred.append(final_emo)
            y_true.append(label_list[i])
        print("Model performance with prompt:", id_prompt)
        compute_metrics(data, y_true, y_pred, probs_emotions, id_prompt, output_file)



def main():
    # Parameters
    data_file = "training.xlsx"
    #data_file = "path_to_your_data_file.tsv"
    output_file = "output_file.xlsx"
    prompts = ["emo_s", "expr_s", "feels_s"]

    template_emo_s = {
    'POS': ['joy', 'achievement', 'pleasure', 'awesome', 'happy', 'blessed'],
    'NEG': ['sadness', 'anger', 'disgust', 'fear', 'shame', 'guilt'],
    'NEU': ['others', 'no emotion']
    }

    template_expr_s = {
    'POS': ['joy', 'an achievement', 'pleasure', 'the awesome', 'happiness', 'the blessing'],
    'NEG': ['sadness', 'anger', 'disgust', 'fear', 'shame', 'guilt'],
    'NEU': ['others', 'no emotion']
    }

    template_feels_s = {
    'POS': ['joyful', 'accomplished', 'pleasure', 'awesome', 'happy', 'blessed'],
    'NEG': ['sadness', 'anger', 'disgusted', 'fear', 'shameful', 'guilty'],
    'NEU': ['others', 'no emotion']
    }


    data = pd.read_excel(data_file)
    compute_entailment(data, template_emo_s, template_expr_s, template_feels_s, prompts, output_file)

if __name__ == "__main__":
    main()


Loading model...
Predicting data using ['emo_s', 'expr_s', 'feels_s'] ...


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2734: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


Model performance with prompt: emo_s
              precision    recall  f1-score   support

         NEG       0.80      0.61      0.69       816
         NEU       0.28      0.15      0.20       240
         POS       0.10      0.50      0.17        76

    accuracy                           0.51      1132
   macro avg       0.39      0.42      0.35      1132
weighted avg       0.64      0.51      0.55      1132



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2734: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


Model performance with prompt: expr_s
              precision    recall  f1-score   support

         NEG       0.77      0.80      0.78       816
         NEU       0.38      0.15      0.21       240
         POS       0.11      0.30      0.17        76

    accuracy                           0.62      1132
   macro avg       0.42      0.41      0.39      1132
weighted avg       0.65      0.62      0.62      1132



/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:2734: FutureWarning: The `truncation_strategy` argument is deprecated and will be removed in a future version, use `truncation=True` to truncate examples to a max length. You can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to truncate to the maximal input size of the model (e.g. 512 for Bert).  If you have pairs of inputs, you can give a specific truncation strategy selected among `truncation='only_first'` (will only truncate the first sentence in the pairs) `truncation='only_second'` (will only truncate the second sentence in the pairs) or `truncation='longest_first'` (will iteratively remove tokens from the longest sentence in the pairs).
  warnings.warn(


Model performance with prompt: feels_s
              precision    recall  f1-score   support

         NEG       0.75      0.81      0.78       816
         NEU       0.31      0.12      0.17       240
         POS       0.13      0.26      0.17        76

    accuracy                           0.63      1132
   macro avg       0.40      0.40      0.37      1132
weighted avg       0.62      0.63      0.61      1132

